In [61]:
import sys
import copy
sys.path.append("/home/elvis/ACDC-UD")
import estrutura_ud
from ufal.udpipe import Model, Pipeline, ProcessingError
corpora = {
    'Petroles': "../Petroles v1.conllu",
    'TOK': "../TOK/TOK_golden.conllu"
}
golden = estrutura_ud.Corpus(recursivo=False)
golden_text = []
for corpus in corpora:
    estrutura = estrutura_ud.Corpus(recursivo=False)
    with open(corpora[corpus]) as f:
        estrutura.build(f.read().replace("yyy", "").replace("zzz", ""))
    corpora[corpus] = copy.deepcopy(estrutura)
    for sentence in corpora[corpus].sentences.values():
        for token in sentence.tokens:
            token.lemma = "_"
            token.upos = "_"
            token.xpos = "_"
            token.feats = "_"
            token.dephead = "_"
            token.deprel = "_"
            token.deps = "_"
            token.misc = "_"
        golden_text.append(sentence.text)
    golden.sentences.update(corpora[corpus].sentences)
model = Model.load("../udpipe/bosqueud_2.5_workbench.udpipe")
pipeline = Pipeline(model, "tokenize", Pipeline.DEFAULT, Pipeline.DEFAULT, "conllu")
pipeline_tokenized = Pipeline(model, "conllu", Pipeline.DEFAULT, Pipeline.DEFAULT, "conllu")
system = estrutura_ud.Corpus(recursivo=False)
system.build(pipeline.process(" ".join(golden_text)))
golden.save("golden.conllu")
os.system("python3 /home/elvis/ACDC-UD/tokenizar_conllu.py golden.conllu golden_tokenized.conllu")
os.system("python3 /home/elvis/ACDC-UD/udpipe_vertical.py ../udpipe/bosqueud_2.5_workbench.udpipe golden_tokenized.conllu golden_udpipe.conllu")
golden_tagged = estrutura_ud.Corpus(recursivo=False)
golden_tagged.load("golden_udpipe.conllu")
os.system("rm golden.conllu golden_tokenized.conllu golden_udpipe.conllu")

build: 1.6685702800750732build: 0.5897412300109863build: 126.19385051727295build: 4.1335179805755615

0

In [62]:
same_text = set(golden_text).intersection([sentence.text for sentence in system.sentences.values()])
same_tokenization = 0
for g in golden_tagged.sentences.values():
    for s in system.sentences.values():
        if g.text == s.text:
            if len(g.tokens) == len(s.tokens):
                same_tokenization += 1
            break
g_different_tokens = set()
for sentence in golden_tagged.sentences.values():
    for token in sentence.tokens:
        if not '-' in token.id:
            g_different_tokens.add(token.word.lower())
s_different_tokens = set()
for sentence in system.sentences.values():
    for token in sentence.tokens:
        if not '-' in token.id:
            s_different_tokens.add(token.word.lower())
g_no_verb = 0
s_no_verb = 0
for sentence in golden_tagged.sentences.values():
    has_verb = False
    for token in sentence.tokens:
        if token.upos == "VERB":
            has_verb = True
            break
    if not has_verb:
        g_no_verb += 1
for sentence in system.sentences.values():
    has_verb = False
    for token in sentence.tokens:
        if token.upos == "VERB":
            has_verb = True
            break
    if not has_verb:
        s_no_verb += 1
g_freq_one = []
g_freq_more = []
s_freq_one = []
s_freq_more = []
for sentence in golden_tagged.sentences.values():
    for token in sentence.tokens:
        if token.word.lower() in g_freq_one:
            g_freq_one.remove(token.word.lower())
            g_freq_more.append(token.word.lower())
        elif token.word.lower() not in g_freq_more:
            g_freq_one.append(token.word.lower())
for sentence in system.sentences.values():
    for token in sentence.tokens:
        if token.word.lower() in s_freq_one:
            s_freq_one.remove(token.word.lower())
            s_freq_more.append(token.word.lower())
        elif token.word.lower() not in s_freq_more:
            s_freq_one.append(token.word.lower())
print("Golden - Sistema\nSentenças: {} - {}\nTokens: {} - {}\nPalavras: {} - {}\nTipos: {} - {}\nTokens com frequência 1: {} - {}\nSem VERB: {} - {}\nSentenças iguais: {}\nTokenização igual: {}".format(
    len(golden_tagged.sentences), len(system.sentences),
    len([token for token in sentence.tokens for sentence in golden_tagged.sentences.values() if not '-' in token.id]), len([token for token in sentence.tokens for sentence in system.sentences.values() if not '-' in token.id]),
    len([token.word.lower() for token in sentence.tokens for sentence in golden_tagged.sentences.values() if token.upos != "PUNCT" and not '-' in token.id]), len([token.word.lower() for token in sentence.tokens for sentence in system.sentences.values() if token.upos != "PUNCT" and not '-' in token.id]),
    len(g_different_tokens), len(s_different_tokens),
    len(g_freq_one), len(s_freq_one),
    g_no_verb, s_no_verb,
    len(same_text),
    same_tokenization,
))

Golden - Sistema
Sentenças: 1140 - 1314
Tokens: 115140 - 132714
Palavras: 103740 - 119574
Tipos: 6437 - 6441
Tokens com frequência 1: 3663 - 3663
Sem VERB: 138 - 241
Sentenças iguais: 934
Tokenização igual: 810
